In [1]:
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
#
#  Failų vardai dabar rodo timestamp
# Skriptas suranda visus json failus užduotame buferiniame folderyje db_path_suppl, patikrina, ar jų nėra 
# vietinėje talpykloje (folderis db_path) ir tada juos perrašo į db_path, papildydamas sarašus list.json, list.xlsx.
# Sarašas saugomas json formatu, gali būti rankomis redaguojamas, jame yra požymis 'incl', 
# kuris lygus 1, jei failas įtraukiamas į analizę. Taip pat yra papildomi flagai F1, F2, F3 kitiems tikslams.

import os, shutil, sys
from pathlib import Path
import pandas as pd
import json

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'ZIVE_BUFFER_2022_03_24'

# Duomenų buferio aplankas
db_folder_suppl = 'buffer'

# ///////////////////////////////////////////////////////////////////////////////////////////

print('\nSkriptas papildo vietinę ZIVE talpyklą duomenimis, esančiais buferyje')

# Nuorodos į aplankus su EKG duomenų rinkiniu ir duomenų buferiu
db_path_suppl = Path(Duomenu_aplankas, db_folder, db_folder_suppl) # Buferis, iš kurio kaupiama
db_path = Path(Duomenu_aplankas, db_folder, 'records') # vietinė talpykla su jau esamais duomenimis arba tuščia

print( 'Duomenų buferio aplankas:', db_path_suppl)
print( 'Vietinės talpyklos aplankas:', db_path)

# - paruošiame dataframe įrašų sužymėjimui
df_list_new = pd.DataFrame({
                        'file_name': pd.Series(dtype='str'),
                        'userId': pd.Series(dtype='str'),
                        'recordingId': pd.Series(dtype='str'),
                        'N': pd.Series(dtype='int'),
                        'S': pd.Series(dtype='int'),
                        'V': pd.Series(dtype='int'),
                        'U': pd.Series(dtype='int'),
                        'incl': pd.Series(dtype='int'), 
                        'flag': pd.Series(dtype='int'),
                        'recorded_at':pd.Series(dtype='datetime64[ns]')})

pd.set_option("display.max_rows", 6000, "display.max_columns", 16)
pd.set_option('display.width', 1000)

# Nuskaitomas, jei yra, jau egzistuojantis sęrašas
list_exists = False
filepath = Path(db_path,'list.json') 
if os.path.isfile(filepath):
    list_exists = True
    df_list = pd.read_json(filepath, orient = 'table')
    df_len = len(df_list)
    print(f'\nTurimas Zive EKG įrašų rinkinys: {db_path}  įrašų kiekis: {df_len}\n')
    # print(df_list)
    nr = df_len
else:
    # Jei nėra, bus kuriamas naujas sąrašas
    print(f'\nSukuriamas naujas Zive EKG įrašų rinkinys aplanke: {db_path}\n')
    nr = 0


# - surenkame aplankų ir failų sąrašą
# https://careerkarma.com/blog/python-list-files-in-directory/

# Walking a directory tree and printing the names of the directories and files
file_list = []
filepaths =[]
new_files = 0

# Surandame buferyje visus failus, kurie neturi extension json
for dirpath, dirnames, files in os.walk(db_path_suppl):
    # https://linuxhint.com/python-os-walk-example/
    # print(dirpath, dirnames, files)
    for file_name in files:
        root, extension = os.path.splitext(file_name)
        if (extension != '.json'):
            # print(file_name)
            filepath = Path(dirpath,file_name)
            filepaths.append(str(filepath))

# Einame per visus buferyje esančius įrašus, įtraukiame į vietinę talpyklą, jei jų ten nėra
for filepath in filepaths:
    filename = os.path.basename(filepath)
    timestamp = int(filename.replace('.', ''))
    # recorded_at = datetime.fromtimestamp(timestamp, tz=timezone.utc)
          
    filepath = Path(filepath + '.json')
    with open(filepath,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    dict_annot = data['rpeakAnnotationCounts']
    df_row = {'file_name':filename, 'userId':data['userId'], 'recordingId':data['recordingId'],
    'N':dict_annot.get('N',0), 'S':dict_annot.get('S',0), 'V':dict_annot.get('V',0),
    'U':dict_annot.get('U',0),'incl':0,'flag':0,'recorded_at':timestamp}
    
    flag_include = False
    if (list_exists == False): # sąrašas neegzistuoja, įtraukiame visus į naują sąrašą
        flag_include = True
    else:   # Sąrašas egzistuoja, įtraukiame į naują sąrašą tik tuos, kurių nėra sename
        if filename not in  df_list['file_name'].values:    
            flag_include = True
    
    if flag_include == True:    
        # df_list = df_list.append(df_row, ignore_index=True)
        df_list_new = df_list_new.append(df_row, ignore_index=True)
        nr +=1
        new_files +=1
        src = Path(db_path_suppl, filename)
        dst = Path(db_path, filename)

        # kopijuojame duomenis ir .json failus, išsaugodami metaduomenis su copy2
        shutil.copy2(src, dst)
        src = Path(db_path_suppl, filename + '.json')
        dst = Path(db_path, filename + '.json')
        shutil.copy2(src, dst)
        
print(f'\nRasta naujų įrašų buferyje: {db_path_suppl}: {new_files}')
if (new_files != 0):
    df_list_new['recorded_at'] =  pd.to_datetime(df_list_new['recorded_at'], unit='s')
    df_list_new = df_list_new.convert_dtypes() # koreguoju dtypes į geriau atitinkančius turinį
    df_list_new.sort_values(by='recorded_at', ascending=True, inplace = True)
    df_list_new.reset_index()
    file_path = Path(db_path,'list_new.json')       
    df_list_new.to_json(file_path, orient = 'table', index=False)
    # df_list_new = pd.read_json(file_path, orient = 'table')
    # Bandžiau įrašyti ir nuskaityti įrašų papildymo sąrašą prieš spausdindamas, 
    # nes nuskaičius json integer timestamp pavirsta datetime. Tačiau
    # to nebereikia, nes paverčiau su pd.to_datime
    print(f'\nNaujų įrašų sąrašas\n')
    print(df_list_new)

if (list_exists == True): # jei sąrašas jau egzistuoja, siujungiame su nauju
    df_list =  pd.concat([df_list,df_list_new], ignore_index=True)
else:
    df_list = df_list_new
    list_exists= True

df_list.sort_values(by='recorded_at', ascending=True, inplace = True)
# print(df_list.dtypes)

if new_files != 0:
    file_path = Path(db_path,'list.json')       
    df_list.to_json(file_path, orient = 'table', index=False)
    print(f'\nAtnaujintas failų sąrašas įrašytas:  {file_path}')

    file_path = Path(db_path,'list.csv') 
    df_list.to_csv(file_path)
    print(f'Atnaujintas failų sąrašas įrašytas į:  {file_path}')    

if (list_exists):
    filepath = Path(db_path,'list.json') 
    df_list = pd.read_json(filepath, orient = 'table')
    df_len = len(df_list)
    print(f'\nZive EKG įrašų rinkinys po papildymo: {db_path}  įrašų kiekis: {df_len}\n')
    print(df_list)


OS in my system :  linux

Skriptas papildo vietinę ZIVE talpyklą duomenimis, esančiais buferyje
Duomenų buferio aplankas: /home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/buffer
Vietinės talpyklos aplankas: /home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/records

Sukuriamas naujas Zive EKG įrašų rinkinys aplanke: /home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/records


Rasta naujų įrašų buferyje: /home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/buffer: 4

Naujų įrašų sąrašas

     file_name                    userId               recordingId    N   S    V  U  incl  flag         recorded_at
0  1630844.567  6144c5b1bd0cc5a681275363  6145aeb6bd0cc5e8392757d1  903   0    0  0     0     0 2021-09-05 12:22:47
1  1631076.286  6144c4fbbd0cc552e427535f  6144cd8bbd0cc50b5e2753b6  541   0  194  0     0     0 2021-09-08 04:44:46
3  1631134.867  613b1d673d08d4d1f3cdc8f8  613b22d43d08d407b7cdc9df  656  12    0  0     0     0 2021-09-08 21:01:07
2  1636452.484  6190d4b23cd1d29db2303ce9

/tmp/ipykernel_1988/1326942438.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list_new = df_list_new.append(df_row, ignore_index=True)
/tmp/ipykernel_1988/1326942438.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list_new = df_list_new.append(df_row, ignore_index=True)
/tmp/ipykernel_1988/1326942438.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list_new = df_list_new.append(df_row, ignore_index=True)
/tmp/ipykernel_1988/1326942438.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_list_new = df_list_new.append(df_row, ignore_index=True)
